In [2]:
import json
import numpy as np
from collections import Counter, defaultdict

In [3]:
#Make sure that the training_corpus.json file is in the same folder as this notebook.
# When running for the first time, generated_training_corpus.json and generated_test_corpus.json do not exist yet.
# They will be generated by running the code in the block below this one.
original_path = 'training_corpus.json' #Original json file
train_path = 'generated_training_corpus.json' #Used for storing generated training set (80%)
test_path = 'generated_test_corpus.json' # Used for storing generated test set (20%)

In [4]:
from random import randint
#Need this to create random samples, hope this is ok.

def split_training_test(train_percent):
    #Splits the training set into a training set and test set.
    #The input variable train_percent determines how large the training set is compared to the test set.
    #For example: train_percent = 80.0 -> 80% of the sentences go to the training set, 20% to the test set.
    train_sentences = json.load(open(original_path, 'r'))
    test_sentences = []
    test_target = int(len(train_sentences) * (1 - train_percent/100))
    for i in range(test_target):
        n = randint(0, len(train_sentences)-1)
        test_sentences.append(train_sentences.pop(n))
    print("Done splitting the sentences.")
    print("Train set length: " , len(train_sentences))
    print("Test set length: " , len(test_sentences))
    print("Writing the files starts now...")
    
    with open(train_path, 'w') as train_file:
        json.dump(train_sentences, train_file)
    print("Done writing the training set!")
    with open(test_path, 'w') as test_file:
        json.dump(test_sentences, test_file)
    print("Done writing the test set!")

#Run the following command to generate a new training and test set:
split_training_test(99.9) 
# Ideal ratio for training set would probably be around 80/20, 
# but larger test set means longer calculation for perplexity.
# To speed up perplexity calculation, I set the train_percent to 99 
#                                               (so test set is only 1% of the data, = 5.000 sentences)

Done splitting the sentences.
Train set length:  495629
Test set length:  496
Writing the files starts now...
Done writing the training set!
Done writing the test set!


In [5]:
class Corpus(object):
    
    """
    This class creates a corpus object read off a .json file consisting of a list of lists,
    where each inner list is a sentence encoded as a list of strings.
    """
    
    def __init__(self, path, t, n, bos_eos=True, vocab=None):
        
        """
        DON'T TOUCH THIS CLASS! 
        IT'S HERE TO SHOW THE PROCESS, YOU DON'T NEED TO ANYTHING HERE. 
        
        A Corpus object has the following attributes:
         - vocab: set or None (default). If a set is passed, words in the input file not 
                         found in the set are replaced with the UNK string
         - path: str, the path to the .json file used to build the corpus object
         - t: int, words with frequency count < t are replaced with the UNK string
         - ngram_size: int, 2 for bigrams, 3 for trigrams, and so on.
         - bos_eos: bool, default to True. If False, bos and eos symbols are not 
                     prepended and appended to sentences.
         - sentences: list of lists, containing the input sentences after lowercasing and 
                         splitting at the white space
         - frequencies: Counter, mapping tokens to their frequency count in the corpus
        """
        
        self.vocab = vocab        
        self.path = path
        self.t = t
        self.ngram_size = n
        self.bos_eos = bos_eos
        
        self.sentences = self.read()
        # output --> [['i', 'am', 'home' '.'], ['you', 'went', 'to', 'the', 'park', '.'], ...]
    
        self.frequencies = self.freq_distr()
        # output --> Counter('the': 485099, 'of': 301877, 'i': 286549, ...)
        # the numbers are made up, they aren't the actual frequency counts
        
        if self.t or self.vocab:
            # input --> [['i', 'am', 'home' '.'], ['you', 'went', 'to', 'the', 'park', '.'], ...]
            self.sentences = self.filter_words()
            # output --> [['i', 'am', 'home' '.'], ['you', 'went', 'to', 'the', 'UNK', '.'], ...]
            # supposing that park wasn't frequent enough or was outside of the training 
            # vocabulary, it gets replaced by the UNK string
            
        if self.bos_eos:
            # input --> [['i', 'am', 'home' '.'], ['you', 'went', 'to', 'the', 'park', '.'], ...]
            self.sentences = self.add_bos_eos()
            # output --> [['bos', i', 'am', 'home' '.', 'eos'], 
            #             ['bos', you', 'went', 'to', 'the', 'park', '.', 'eos'], ...]
                    
    def read(self):
        
        """
        Reads the sentences off the .json file, replaces quotes, lowercases strings and splits 
        at the white space. Returns a list of lists.
        """
        
        if self.path.endswith('.json'):
            sentences = json.load(open(self.path, 'r'))                
        else:   
            sentences = []
            with open(self.path, 'r', encoding='latin-1') as f:
                for line in f:
                    print(line[:20])
                    # first strip away newline symbols and the like, then replace ' and " with the empty 
                    # string and get rid of possible remaining trailing spaces 
                    line = line.strip().translate({ord(i): None for i in '"\'\\'}).strip(' ')
                    # lowercase and split at the white space (the corpus has ben previously tokenized)
                    sentences.append(line.lower().split(' '))
        
        return sentences
    
    def freq_distr(self):
        
        """
        Creates a counter mapping tokens to frequency counts
        
        count = Counter()
        for sentence in self.sentences:
            for word in sentence:
                count[w] += 1
            
        """
    
        return Counter([word for sentence in self.sentences for word in sentence])
        
    
    def filter_words(self):
        
        """
        Replaces illegal tokens with the UNK string. A token is illegal if its frequency count
        is lower than the given threshold and/or if it falls outside the specified vocabulary.
        The two filters can be both active at the same time but don't have to be. To exclude the 
        frequency filter, set t=0 in the class call.
        """
        
        filtered_sentences = []
        for sentence in self.sentences:
            filtered_sentence = []
            for word in sentence:
                if self.t and self.vocab:
                    # check that the word is frequent enough and occurs in the vocabulary
                    filtered_sentence.append(
                        word if self.frequencies[word] > self.t and word in self.vocab else 'UNK'
                    )
                else:
                    if self.t:
                        # check that the word is frequent enough
                        filtered_sentence.append(word if self.frequencies[word] > self.t else 'UNK')
                    else:
                        # check if the word occurs in the vocabulary
                        filtered_sentence.append(word if word in self.vocab else 'UNK')
                        
            if len(filtered_sentence) > 1:
                # make sure that the sentence contains more than 1 token
                filtered_sentences.append(filtered_sentence)
    
        return filtered_sentences
    
    def add_bos_eos(self):
        
        """
        Adds the necessary number of BOS symbols and one EOS symbol.
        
        In a bigram model, you need one bos and one eos; in a trigram model you need two bos and one eos, 
        and so on...
        """
        
        padded_sentences = []
        for sentence in self.sentences:
            padded_sentence = ['#bos#']*(self.ngram_size-1) + sentence + ['#eos#']
            padded_sentences.append(padded_sentence)
    
        return padded_sentences

In [7]:
class LM(object):
    
    """
    Creates a language model object which can be trained and tested.
    The language model has the following attributes:
     - vocab: set of strings
     - lam: float, indicating the constant to add to transition counts to smooth them (default to 1)
     - ngram_size: int, the size of the ngrams
    """
    
    def __init__(self, n, vocab=None, smooth='Laplace', lam=1, lambdas = [-1]):
        
        self.vocab = vocab
        self.lam = lam
        self.ngram_size = n
        self.smooth = smooth
        self.counts = [-1]
        if not smooth == 'Laplace':
            #If we are not using LaPlace smoothing, that means we want to use discounting.
            #To do this, we need lambda values to determine the weight of different ngram sizes.
            self.lambdas = lambdas
            #lambdas contains a list of weights, all add up to 1.
            #The exact values of the weights need to be finetuned.
            #lambda[0] is used as the weight for ngram_size == 1 (target frequency)
            #lambda[1] is used as the weight for ngram_size == 2 (history of size 1)
            #etc..
        
    def get_ngram(self, sentence, i, size=-1):
        
        """
        CHANGE AT OWN RISK.
        
        Takes in a list of string and an index, and returns the history and current 
        token of the appropriate size: the current token is the one at the provided 
        index, while the history consists of the n-1 previous tokens. If the ngram 
        size is 1, only the current token is returned.
        
        Example:
        input sentence: ['bos', 'i', 'am', 'home', 'eos']
        target index: 2
        ngram size: 3
        
        ngram = ['bos', 'i', 'am']  
        #from index 2-(3-1) = 0 to index i (the +1 is just because of how Python slices lists) 
        
        history = ('bos', 'i')
        target = 'am'
        return (('bos', 'i'), 'am')
        """
        # We have added the size parameter to this function. 
        # This can be used to request ngrams of different sizes (useful for discounting)
        # In case the size parameter is not explicitly defined, the following if statement sets it to self.ngram_size
        #  so that the function still works like it did before our modification.
        if size == -1:
            size = self.ngram_size
        
        if size == 1:
            return sentence[i]
        else:
            ngram = sentence[i-(size-1):i+1]
            history = tuple(ngram[:-1])
            target = ngram[-1]
            return (history, target)
    
    def update_multiple_counts(self, corpusList):
        # This function is used to call update_counts multiple times, 
        #  when multiple counts matrices are needed for different ngram sizes (for discounting)
        n = len(corpusList)
        self.counts = [-1]*n
        for i in range(n):
            self.update_counts(corpusList[i], i)
            
            
    
    def update_counts(self, corpus, corpusIndex = 0):
        
        """
        CHANGE AT OWN RISK.
        
        Creates a transition matrix with counts in the form of a default dict mapping history
        states to current states to the co-occurrence count (unless the ngram size is 1, in which
        case the transition matrix is a simple counter mapping tokens to frequencies. 
        The ngram size of the corpus object has to be the same as the language model ngram size.
        The input corpus (passed by providing the corpus object) is processed by extracting ngrams
        of the chosen size and updating transition counts.
        
        This method creates three attributes for the language model object:
         - counts: dict, described above
         - vocab: set, containing all the tokens in the corpus
         - vocab_size: int, indicating the number of tokens in the vocabulary
        """
        #Commented out the following section for now, as this check does not work with the new LM that uses mutliple counts matrices.
        """if self.ngram_size != corpus.ngram_size:
            raise ValueError("The corpus was pre-processed considering an ngram size of {} while the "
                             "language model was created with an ngram size of {}. \n"
                             "Please choose the same ngram size for pre-processing the corpus and fitting "
                             "the model.".format(corpus.ngram_size, self.ngram_size))"""
        
        self.counts[corpusIndex] = defaultdict(dict) if corpus.ngram_size > 1 else Counter()
        for sentence in corpus.sentences:
            for idx in range(corpus.ngram_size-1, len(sentence)):
                ngram = self.get_ngram(sentence, idx, corpus.ngram_size)
                if corpus.ngram_size == 1:
                    self.counts[corpusIndex][ngram] += 1
                else:
                    # it's faster to try to do something and catch an exception than to use an if statement to check
                    # whether a condition is met beforehand. The if is checked everytime, the exception is only catched
                    # the first time, after that everything runs smoothly
                    try:
                        self.counts[corpusIndex][ngram[0]][ngram[1]] += 1
                    except KeyError:
                        self.counts[corpusIndex][ngram[0]][ngram[1]] = 1
        
        # first loop through the sentences in the corpus, than loop through each word in a sentence
        self.vocab = {word for sentence in corpus.sentences for word in sentence}
        self.vocab_size = len(self.vocab)
    
    def get_unigram_probability(self, ngram):
        
        """
        CHANGE THIS.
        
        Compute the probability of a given unigram in the estimated language model using
        Laplace smoothing (add k). 
        """
        # We have added an if statement that handles this function differently when the smoothing method is not 
        #  set to 'Laplace'. The alternative does not use add-K smoothing, but if somehow the ngram count still
        #  results in 0, we add 0.000001
        if self.smooth == 'Laplace':
            tot = sum(list(self.counts[0].values())) + (self.vocab_size*self.lam)
            try:
            #This generally triggers if the history-list exists in the counts matrix
                ngram_count = self.counts[0][ngram] + self.lam
            except KeyError:
            #This exception triggers if the history-list does not exist in the counts matrix. 
            #This should never happen as all UNIgrams either appear in the counts matrix or are replaced by UNK.
                ngram_count = self.lam
                print(ngram_count, tot)

            return ngram_count/tot
        else: #If self.smooth is not Laplace, apply discounting. 
            #   For unigram, that means just using the frequency of the unigram, without adding k.
            tot = sum(list(self.counts[0].values()))
            try:
                ngram_count = self.counts[0][ngram]
                if ngram_count == 0:
                    ngram_count = 0.000001
            except KeyError:
                #This should never happen, as we were promised that all words are in the vocabulary 
                # (and thus in the unigram counts matrix)
                ngram_count = 0.000001
                print("Error, word not in vocabulary")
            return ngram_count/tot

    
    def get_ngram_probability(self, history, target):
        """
        IF self.smooth == 'Laplace': 
            Compute the conditional probability of the target token given the history, 
                using Laplace smoothing (add k).
        ELSE:
            Compute the conditional probability of the target token given the history, 
                using Discounting. This means computing the conditional probability of the target token given the history,
                and multiplying this value by its lambda weight. This is repeated recursively with smalller and smaller
                history lists (history[1:]), until len(history) is 0, indicating that a unigram is reached. Finally,
                get_unigram_probability is called once to find the unigram probability, and all scores are combined.
        """
        if self.smooth == 'Laplace':
            try:
                #This generally triggers if the history-list exists in the counts matrix
                ngram_tot = np.sum(list(self.counts[0][history].values())) + (self.vocab_size*self.lam)
                #ngram_tot is used to estimate the probability in the last step of this function.
                #it stands for the sum of all frequencies of each engram, + the total added smoothing.
                try:
                    #This try part goes through if both the history and the target exist in the counts matrix.
                    transition_count = self.counts[0][history][target] + self.lam
                except KeyError:
                    #If the history exists but the target does not exist, set transition count to k:
                    transition_count = self.lam
            except KeyError:
                #This exception triggers if the history-list does not exist in the counts matrix
                transition_count = self.lam
                ngram_tot = self.vocab_size*self.lam

            return transition_count/ngram_tot
        else:
            #If smoothing is not set to Laplace, apply Discounting
            history_size = len(history)
            if history_size == 0: #If there was no history, this is a unigram
                return self.lambdas[0] * self.get_unigram_probability(target) 
                #Then return the unigram probability, multiplied by the lambda weight
            else:
                try:
                    #If the history engram does exist in counts:
                    ngram_tot = np.sum(list(self.counts[history_size][history].values()))
                    try: 
                        #And if the target also exists in counts:
                        transition_count = self.counts[history_size][history][target]
                    except KeyError:
                        #If the history engram does exist in counts, but target does not:
                        transition_count = 0
                except KeyError:
                    #If the history engram does not exist in counts:
                    transition_count = 0
                    ngram_tot = 1
            if ngram_tot == 0:
                transition_count = 0
                ngram_tot = 1
            #Calculate the probability of target appearing after history based only on this ngram size:
            ngram_prob = transition_count/ngram_tot 
            #Multiply it by the corresponding weight, and add the weighted scores of the lower ngram sizes:
            return self.lambdas[history_size] * ngram_prob + self.get_ngram_probability(history[1:], target)
                
            
    def setLambdas(self, lambdas):
        # Used to change the lambdas without having to re-count the counts matrices, or rebuild corpi, etc.
        self.lambdas = lambdas
    
    def perplexity(self, test_corpus):
        """
        Uses the estimated language model to process a corpus and computes the perplexity 
        of the language model over the corpus.
        
        DON'T TOUCH THIS FUNCTION!!!
        """
        
        probs = []
        for sentence in test_corpus.sentences:
            for idx in range(self.ngram_size-1, len(sentence)):
                ngram = self.get_ngram(sentence, idx)
                if self.ngram_size == 1:
                    probs.append(self.get_unigram_probability(ngram))
                else:
                    probs.append(self.get_ngram_probability(ngram[0], ngram[1]))
        entropy = np.log2(probs)
        # this assertion makes sure that you retrieved valid probabilities, whose log must be <= 0
        assert all(entropy <= 0)
        
        avg_entropy = -1 * (sum(entropy) / len(entropy))
        
        return pow(2.0, avg_entropy)

In [45]:
# example code to run a unigram model with add 0.001 smoothing. Tokens with a frequency count lower than 10
# are replaced with the UNK string
n = 1
train_corpus = Corpus(train_path, 10, n, bos_eos=True, vocab=None)
print("Corpus created..")
unigram_model = LM(n, lam=0.001)
print("Language Model created..")
unigram_model.update_counts(train_corpus)
print("Counts updated..")

# to ensure consistency, the test corpus is filtered using the vocabulary of the trained language model
test_corpus = Corpus(test_path, None, n, bos_eos=True, vocab=unigram_model.vocab)
print("Test_corpus created..")
p = unigram_model.perplexity(test_corpus)
print("Perplexity calculated! Perplexity of the unigram model: {}".format(p))

Corpus created..
Language Model created..
Counts updated..
Test_corpus created..
Perplexity calculated! Perplexity of the unigram model: 816.230178051716


In [46]:
# Calculating perplexity for the same model, but using discounting:
#Note, this should be roughly the same, as no discounting can occur with an engram size of 1
n = 1
train_corpus = Corpus(train_path, 10, n, bos_eos=True, vocab=None)
print("Corpus created..")
unigram_model = LM(n, lam=0.001, smooth = 'discount', lambdas = [1])
print("Language Model created..")
unigram_model.update_counts(train_corpus)
print("Counts updated..")

# to ensure consistency, the test corpus is filtered using the vocabulary of the trained language model
test_corpus = Corpus(test_path, None, n, bos_eos=True, vocab=unigram_model.vocab)
print("Test_corpus created..")
p = unigram_model.perplexity(test_corpus)
print("Perplexity calculated! Perplexity of the unigram model: {}".format(p))

Corpus created..
Language Model created..
Counts updated..
Test_corpus created..
Perplexity calculated! Perplexity of the unigram model: 816.2301205179506


In [47]:
# example code to run a bigram model with add 0.001 smoothing. The same frequency threshold is applied.
n = 2
train_corpus = Corpus(train_path, 10, n, bos_eos=True, vocab=None)
print("Corpus created..")
bigram_model = LM(n, lam=0.001)
print("Language Model created..")
bigram_model.update_counts(train_corpus)
print("Counts updated..")

# to ensure consistency, the test corpus is filtered using the vocabulary of the trained language model
test_corpus = Corpus(test_path, None, n, bos_eos=True, vocab=bigram_model.vocab)
print("Test_corpus created..")
p = bigram_model.perplexity(test_corpus)
print("Perplexity calculated! Perplexity of the bigram model: {}".format(p))

Corpus created..
Language Model created..
Counts updated..
Test_corpus created..
Perplexity calculated! Perplexity of the bigram model: 293.21398546789186


In [48]:
n = 2
train_corpus_list = []
for i in range(n):
    train_corpus_list.append(Corpus(train_path, 10, i+1, bos_eos=True, vocab = None))
print("Corpus list created..")
bigram_model = LM(n, lam=0.001, smooth = 'discount', lambdas = [0.2, 0.8])
print("Language Model created..")

bigram_model.update_multiple_counts(train_corpus_list)
print("Counts updated..")

# to ensure consistency, the test corpus is filtered using the vocabulary of the trained language model
test_corpus = Corpus(test_path, None, n, bos_eos=True, vocab=bigram_model.vocab)
print("Test_corpus created..")
p = bigram_model.perplexity(test_corpus)
print("Perplexity calculated! Perplexity of the bigram model: {}".format(p))

Corpus list created..
Language Model created..
Counts updated..
Test_corpus created..
Perplexity calculated! Perplexity of the bigram model: 239.6711354689498


In [49]:
# example code to run a trigram model with add 0.001 smoothing. The same frequency threshold is applied.
n = 3
train_corpus = Corpus(train_path, 10, n, bos_eos=True, vocab=None)
print("Corpus created..")
trigram_model = LM(n, lam=0.001)
print("Language Model created..")
trigram_model.update_counts(train_corpus)
print("Counts updated..")

# to ensure consistency, the test corpus is filtered using the vocabulary of the trained language model
test_corpus = Corpus(test_path, None, n, bos_eos=True, vocab=trigram_model.vocab)
print("Test_corpus created..")
p = trigram_model.perplexity(test_corpus)
print("Perplexity calculated! Perplexity of the trigram model: {}".format(p))

Corpus created..
Language Model created..
Counts updated..
Test_corpus created..
Perplexity calculated! Perplexity of the trigram model: 988.3821617822591


In [51]:
n = 3
train_corpus_list = []
for i in range(n):
    train_corpus_list.append(Corpus(train_path, 10, i+1, bos_eos=True, vocab = None))
print("Corpus list created..")
trigram_model = LM(n, lam=0.001, smooth = 'discount', lambdas = [0.225, 0.55, 0.225]) #0.1, 0.3, 0.6 -> 222 perplexity
print("Language Model created..")

trigram_model.update_multiple_counts(train_corpus_list)
print("Counts updated..")

# to ensure consistency, the test corpus is filtered using the vocabulary of the trained language model
test_corpus = Corpus(test_path, None, n, bos_eos=True, vocab=trigram_model.vocab)
print("Test_corpus created..")
p = trigram_model.perplexity(test_corpus)
print("Perplexity calculated! Perplexity of the trigram model: {}".format(p))

Corpus list created..
Language Model created..
Counts updated..
Test_corpus created..
Perplexity calculated! Perplexity of the trigram model: 200.7484293639936


### Best performing model:
###### Building the corpi:

In [52]:
n = 5
train_corpus_list = []
for i in range(n):
    train_corpus_list.append(Corpus(train_path, 10, i+1, bos_eos=True, vocab = None))
    print("Corpus {} out of {} has been created..".format(i+1, n))
print("Corpus list created..")


Corpus 1 out of 5 has been created..
Corpus 2 out of 5 has been created..
Corpus 3 out of 5 has been created..
Corpus 4 out of 5 has been created..
Corpus 5 out of 5 has been created..
Corpus list created..


###### Creating the language model:

In [53]:
lambdas = [0.15, 0.50, 0.2, 0.1, 0.05]
ngram_model = LM(n, lam=0.001, smooth = 'discount', lambdas = lambdas)
print("Language Model created..")

ngram_model.update_multiple_counts(train_corpus_list)
print("Counts updated..")

# to ensure consistency, the test corpus is filtered using the vocabulary of the trained language model
test_corpus = Corpus(test_path, None, n, bos_eos=True, vocab=ngram_model.vocab)
print("Test_corpus created..")
p = ngram_model.perplexity(test_corpus)
print("Perplexity calculated! Perplexity of the ngram model: {}".format(p))
print("Lambda's used: {}".format(lambdas))

Language Model created..
Counts updated..
Test_corpus created..
Perplexity calculated! Perplexity of the ngram model: 200.82333069237947
Lambda's used: [0.15, 0.5, 0.2, 0.1, 0.05]


###### Finetuning the hyperparameters:

In [54]:
ngram_model.setLambdas([0.18, 0.55, 0.18, 0.05, 0.04])
p = ngram_model.perplexity(test_corpus)
print("Perplexity calculated! Perplexity of the ngram model: {}".format(p))
print("Lambda's used: {}".format(ngram_model.lambdas))

Perplexity calculated! Perplexity of the ngram model: 197.05030981240427
Lambda's used: [0.18, 0.55, 0.18, 0.05, 0.04]


In [31]:
"""
# Checking how unique different length history ngrams are on average:
for n_size in range(1, len(ngram_model.counts)):
    counts_n = ngram_model.counts[n_size]
    keys = counts_n.keys()
    length = len(keys)
    total = 0
    for key in keys:
        history_dict = counts_n[key]
        for hkey in history_dict.keys():
            total += history_dict[hkey]
    print("The average frequency of each history engram in the counts {} matrix is: {}".format(n_size, total/length))
# Output:
# n = 2:
# The average frequency of each history engram in the counts 1 matrix is: 343.01509314808965
# -> Indicating that on average, any history engram of size 1 (single word) appears 343 times in the corpus
# n = 3:
# The average frequency of each history engram in the counts 2 matrix is: 5.9779641672839094
# -> Indicating that on average, any history engram of size 2 (two words) appears ~6 times in the corpus
# n = 4: 
# The average frequency of each history engram in the counts 3 matrix is: 1.9367961181665696
# -> Indicating that on average, any history engram of size 3 appears ~2 times in the corpus
# n = 5:
# The average frequency of each history engram in the counts 4 matrix is: 1.3445868169375572
# -> Indicating that on average, any history engram of size 4 appears ~1.34 times in the corpus
# --> So about 1 in 3 history engrams of size 4 appears more than once (not exactly, but helps to think this way)

# n = 6:
# The average frequency of each history engram in the counts 5 matrix is: 1.2185491853222503
# -> Indicating that on average, any history engram of size 5 appears ~1.22 times in the corpus
# --> So about 1 in 4.6 history engrams of size 4 appears more than once (not exactly, but helps to think this way)
# n = 7:
# The average frequency of each history engram in the counts 6 matrix is: 1.190698265789514
# -> Indicating that on average, any history engram of size 6 appears ~1.2 times in the corpus
# --> So about 1 in 5 history engrams of size 4 appears more than once (not exactly, but helps to think this way)
    
"""

The average frequency of each history engram in the counts 1 matrix is: 343.01509314808965
The average frequency of each history engram in the counts 2 matrix is: 5.9779641672839094
The average frequency of each history engram in the counts 3 matrix is: 1.9367961181665696
The average frequency of each history engram in the counts 4 matrix is: 1.3445868169375572
The average frequency of each history engram in the counts 5 matrix is: 1.2185491853222503
The average frequency of each history engram in the counts 6 matrix is: 1.190698265789514
